In [3]:
# Supaprastintas skriptas rpeaks sekų, gautų iš .json failo ir EKG įrašo, panaudojant NEUROKIT2, sulyginimui 
# Sulyginimas daromas dviem įrašams, kurie yra pridėti:

import pandas as pd
import numpy as np
import json
import neurokit2 as nk

def zive_read_file_1ch(filename):
    f = open(filename, "r")
    a = np.fromfile(f, dtype=np.dtype('>i4'))
    ADCmax=0x800000
    Vref=2.5
    b = (a - ADCmax/2)*2*Vref/ADCmax/3.5*1000
    ecg_signal = b - np.mean(b)
    return ecg_signal

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

# file_names = ['1625402.027', '1625400.796', '1630757.924']
# file_names = ['1630757.924']
file_names = ['1625402.027']
# file_names = ['1625400.796']

for file_name in file_names:
    
    print(f"\nZive įrašas:  {file_name:>2}")

    # I-a dalis: nuskaitome rpeaks iš json failo
    filename = file_name + '.json'
    with open(filename,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    rpeaks_json = pd.DataFrame(data['rpeaks'])['sampleIndex']

    print(f"\rpeaks iš json failo: {len(rpeaks_json)}")
    # print(rpeaks_json) 
    
    # II-a dalis: suformuojame rpeaks su Neurokitu
    # ecg_signal_df = pd.DataFrame(zive_read_file_1ch(str(file_name)), columns=['orig'])
    # file_names = ['1625402.027']
    ecg_signal = zive_read_file_1ch(file_name)
    _, rpeaks = nk.ecg_peaks(ecg_signal, sampling_rate=200, correct_artifacts=False)
    rpeaks_sgn = pd.DataFrame(rpeaks)['ECG_R_Peaks']

    print(f"\rpeaks iš neurokit2: {len(rpeaks_sgn)}")
    # print(rpeaks_sgn)

    # Sulyginame
    
    result = (rpeaks_json == rpeaks_sgn)
    joint_df = pd.concat([rpeaks_json, rpeaks_sgn, result], axis=1)
    
    joint_df.rename(columns={'sampleIndex': 'json', 'ECG_R_Peaks': 'neurokit'}, inplace=True)
    # print(joint_df)

    uniq_values =result.value_counts()
    # print(uniq_values)
    print(f"Sutapo reikšmių: {uniq_values[1]}  nesutapo: {uniq_values[0]}")




Zive įrašas:  1625402.027
peaks iš json failo: 765
peaks iš neurokit2: 765
Sutapo reikšmių: 170  nesutapo: 595
